# CNN for Cifar-10 Dataset

## Introduction

In this first notebook we are going to explain how we have set up our work.  
Essentially the process is divided into several parts:
* Set up environment with the Cifar-10 Dataset
* Define a convolutional neural network
* Define a quantization method
* Train the convolutional neural network
* Provide information about CNN's performance and accuracy

### Requirements

For this project we are used:
* Python 3.6
* Tensorflow 1.8.0
* *GPUs parallel calculation manager* nVidia CUDA 9.0
* *GPU-accelerated library* nVidia cuDNN 7.1
* or CPU optimized tensorflow for intel

In [2]:
import numpy as np
import tensorflow as tf

from cnn.dense import (eval_fn, forward_pass, loss_fn)
from cnn.model_class import TfClassifier
from cnn.utils.dataset import load_cifar10
from cnn.utils.graph_manipulation import MODELS_DIR, load_frozen_graph

## Cifar-10 Dataset

Cifar-10 Dataset is taken from the official website www.cs.toronto.edu.

Dataset is stored in the data directory: cnn/data. From Cifar-10 dataset we are going to take x_train, t_train, x_test and t_test.
The training dataset set is used for training the CNN, the testing dataset is used for evaluate the performance and the accuracy of the network.

In [2]:
x_train, t_train, x_test, t_test = load_cifar10()

In [6]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

It also needs to be normalized

In [3]:
x_train = dataset_preprocessing_by_keras(x_train)
x_train[0, :, :, 0]

array([[-0.68747891, -0.65572952, -0.63985482, ..., -0.95734874,
        -1.00497283, -0.92559935],
       [-0.65572952, -0.63985482, -0.59223073, ..., -0.92559935,
        -1.02084753, -0.86210057],
       [-0.7192283 , -0.63985482, -0.65572952, ..., -0.68747891,
        -0.735103  , -0.70335361],
       ...,
       [-0.36998499, -0.7509777 , -1.00497283, ..., -0.25886212,
        -0.24298742, -0.14773924],
       [-0.25886212, -0.65572952, -1.19546919, ...,  0.31262694,
        -0.00486698, -0.49698256],
       [-0.27473681, -0.35411029, -1.11609571, ...,  0.75711843,
         0.64599556,  0.28087755]])

## CNN Model and Training

We will use a custom made wrapper for tensorfllow NN training and use

This CNN is called *dense_cnn*. Here we will explain how it is composed.

The CNN is composed by several layers. In the first part there are 2 **convolutional** layers and 2 **pooling** layers (they are alternated), then there are a *flatten* layer followed by a **relu** layer, a *dropout* layer and finally a **softmax** layer.

The network uses a stochastic gradient descent optimizer and a categorical crossentropy loss.  
To judge the performance of our model we are used a MSE metric.

In [3]:
model = TfClassifier(NET_NAME, forward_pass, loss_fn, eval_fn,
                     tf.train.AdamOptimizer())

This network is trained for 50 epochs.

In [4]:
history = model.fit(
    [x_train, t_train],
    batch_size=1,
    validation_split=0.1,
    epochs=1,
    verbosity=2,
    keep_prob=0.5)

print(history)

For training: tensorboard --logdir=/tmp/log-tb/dense_cnn/training
For validation: tensorboard --logdir=/tmp/log-tb/dense_cnn/validation
{'classes': array([1]), 'accuracy': (0.0, 0.0), 'mse': (0.0, 1677.4539), 'loss': 79.16019}
{'classes': array([7]), 'accuracy': (0.0, 0.0), 'mse': (1677.4539, 14643.992), 'loss': 471.7913}
{'classes': array([7]), 'accuracy': (0.0, 0.0), 'mse': (14643.992, 20264.693), 'loss': 240.14194}
{'classes': array([5]), 'accuracy': (0.0, 0.0), 'mse': (20264.693, 19044.682), 'loss': 215.25688}
{'classes': array([5]), 'accuracy': (0.0, 0.2), 'mse': (19044.682, 16337.645), 'loss': 0.0}
{'classes': array([5]), 'accuracy': (0.2, 0.16666667), 'mse': (16337.645, 14145.406), 'loss': 133.26247}
{'classes': array([6]), 'accuracy': (0.16666667, 0.14285715), 'mse': (14145.406, 12435.828), 'loss': 92.2276}
{'classes': array([6]), 'accuracy': (0.14285715, 0.125), 'mse': (12435.828, 11080.518), 'loss': 71.087906}
{'classes': array([6]), 'accuracy': (0.125, 0.11111111), 'mse': (1

KeyboardInterrupt: 

Then it's evaluated

In [9]:
evals = model.evaluate([x_test, t_test])

print(evals)

INFO:tensorflow:Restoring parameters from /home/daibak/Documents/Code/Python/aca-tensorflow/cnn/models/dense_cnn/model.ckpt
[{'logits': array([[  58.449165 ,  -40.94816  ,   -5.6226172, ...,  -94.2457   ,
          43.86064  ,  -68.44922  ],
       [  -9.421125 ,   -2.6976483,  -56.005733 , ...,  -15.75792  ,
          16.243078 ,  -80.76974  ],
       [  88.704    ,  -35.250233 ,  -27.641563 , ..., -143.42578  ,
         217.6601   ,   21.607813 ],
       ...,
       [ -35.267845 ,   -7.7263284,  -25.900536 , ...,   25.24029  ,
           5.2269926,  -20.619165 ],
       [  45.99917  ,  119.776566 , -125.03786  , ...,  -61.233307 ,
         179.8653   ,   47.13309  ],
       [  22.152536 ,  -10.786198 ,  -36.33955  , ..., -121.99178  ,
         116.80828  ,   32.390125 ]], dtype=float32), 'classes': array([0, 3, 8, ..., 5, 8, 8]), 'probabilities': array([[9.9999952e-01, 0.0000000e+00, 1.4927161e-28, ..., 0.0000000e+00,
        4.6161975e-07, 0.0000000e+00],
       [3.5268134e-31, 2.93

## Optimization Transform

In [3]:
graph = load_frozen_graph("models/dense_quant.pb")

In [4]:
graph.get_operations()

[<tf.Operation 'features' type=Placeholder>,
 <tf.Operation 'conv2d/Conv2D_eightbit/features__port__0/reduction_dims' type=Const>,
 <tf.Operation 'conv2d/Conv2D_eightbit/features__port__0/reshape_dims' type=Const>,
 <tf.Operation 'conv2d/Conv2D_eightbit/features__port__0/reshape' type=Reshape>,
 <tf.Operation 'conv2d/Conv2D_eightbit/features__port__0/max' type=Max>,
 <tf.Operation 'conv2d/Conv2D_eightbit/features__port__0/min' type=Min>,
 <tf.Operation 'conv2d/Conv2D_eightbit/features__port__0/quantize' type=QuantizeV2>,
 <tf.Operation 'conv2d/weights_quant/FakeQuantWithMinMaxVars/_0__cf__0_quantized_min' type=Const>,
 <tf.Operation 'logits/weights_quant/FakeQuantWithMinMaxVars/_2__cf__2_quantized_max' type=Const>,
 <tf.Operation 'conv2d/bias' type=Const>,
 <tf.Operation 'conv2d/BiasAdd_eightbit/conv2d/bias/reduction_dims' type=Const>,
 <tf.Operation 'conv2d/BiasAdd_eightbit/conv2d/bias/reshape_dims' type=Const>,
 <tf.Operation 'conv2d/BiasAdd_eightbit/conv2d/bias/reshape' type=Reshape